In [1]:
!pip install python-telegram-bot pandas scikit-learn python-dotenv torch transformers huggingface-hub nest-asyncio


In [2]:
from google.colab import files
uploaded = files.upload()


Saving airline_faq.xlsx to airline_faq (6).xlsx


In [3]:
import os
from dotenv import load_dotenv
from google.colab import userdata

# load_dotenv() # No longer needed if using Colab secrets
TELEGRAM_BOT_TOKEN = userdata.get("TELEGRAM_BOT_TOKEN")
HUGGINGFACE_API_TOKEN = os.getenv("HUGGINGFACE_API_TOKEN") # Assuming HF token might still be in env vars or needs to be handled separately

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

faq_df = pd.read_excel("airline_faq.xlsx")
print(faq_df.columns)
faq_df.dropna(subset=["Question", "Answer"], inplace=True)

vectorizer = TfidfVectorizer(stop_words="english")
faq_vectors = vectorizer.fit_transform(faq_df["Question"])

def retrieve_faq(query, top_k=3):
    query_vec = vectorizer.transform([query])
    sims = cosine_similarity(query_vec, faq_vectors).flatten()
    top_idxs = sims.argsort()[-top_k:][::-1]
    return faq_df.iloc[top_idxs]

Index(['Question', 'Answer'], dtype='object')


In [5]:
import torch
from transformers import pipeline
from huggingface_hub import login
from google.colab import userdata

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

HUGGINGFACE_API_TOKEN = userdata.get("HUGGINGFACE_API_TOKEN")
login(token=HUGGINGFACE_API_TOKEN)

pipe = pipeline(
    "text-generation",
    model="tiiuae/falcon-7b-instruct",
    device_map="auto",
    dtype=torch.float16,
    model_kwargs={"temperature": 0.3, "max_length": 200, "do_sample": False}
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [12]:
def generate_answer(user_query):
    top_faqs = retrieve_faq(user_query)
    # Remove the "Q: ... A: ..." format from the context
    context = "\n".join([f"{a}" for q, a in zip(top_faqs['Question'], top_faqs['Answer'])])
    # Modify the prompt to have a more conversational tone and remove "User" reference
    prompt = f"You are AuroraBot, an airline assistant. Use the following relevant information to answer the question in a human-like, conversational manner:\n\nRelevant information:\n{context}\n\nQuestion: {user_query}\nAuroraBot's answer:"
    try:
        result = pipe(prompt, return_full_text=False)[0]["generated_text"]
        return result.strip()
    except Exception as e:
        return "Sorry, I couldn't generate a response right now."

In [13]:
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, ContextTypes, filters
import logging
from google.colab import userdata

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("AuroraBot")

async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text("Hello! I'm AuroraBot 🛫, your friendly airline assistant. How can I help you today?")

async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_msg = update.message.text.lower()  # Convert to lowercase for easier matching
    logger.info(f"User: {user_msg}")

    # Check for simple greetings
    if user_msg in ["hi", "hello", "hey"]:
        response = "Hello there! How can I assist you today?"
    else:
        response = generate_answer(user_msg)

    await update.message.reply_text(response)

async def main():
    telegram_bot_token = userdata.get("TELEGRAM_BOT_TOKEN")
    if not telegram_bot_token:
        logger.error("Telegram bot token not found in Colab secrets.")
        return
    app = ApplicationBuilder().token(telegram_bot_token).build()
    app.add_handler(CommandHandler("start", start))
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))
    await app.bot.delete_webhook(drop_pending_updates=True)
    # Update the logger.info with the correct bot username
    logger.info("AuroraBot is live! Access it here: t.me/AuroraSkiesBot")
    await app.run_polling()

In [ ]:
import nest_asyncio
import asyncio
from telegram.ext import ApplicationBuilder # Import necessary class

nest_asyncio.apply()
loop = asyncio.get_event_loop()

# Check if an application instance exists and shut it down
if 'app' in locals() and isinstance(app, ApplicationBuilder):
    loop.run_until_complete(app.shutdown())


loop.run_until_complete(main())

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
